In [1]:
# from sst and rainfall ---> rainfall
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt 
import tensorflow.compat.v1 as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv3D, Conv2D
from keras.layers import ConvLSTM2D
from keras.layers import Attention
from keras.layers import Input
from keras.models import Model
from keras.layers import Dropout, Lambda
from keras import optimizers
from keras.layers import Conv2DTranspose, TimeDistributed
from keras import callbacks
import xarray as xr

# Define global variables
lead_time = 1
save_path = "/home/cccr/roxy/matin/MTech_project/model/Conv-LSTM/7in2out/"
model_path = save_path + "ConvLstm_sst13rf_rf_1bob.h5"
add_data = "/home/cccr/roxy/matin/MTech_project/data/"

def preprocess_data(sequence, n_steps, channels):
    """
    Function to preprocess the data and prepare it for model training.
    
    Parameters:
        sequence (xarray.Dataset): Data sequence to be preprocessed.
        n_steps (int): Number of time steps in the input sequence.
        channels (int): Number of channels in the input sequence.
    
    Returns:
        tuple: A tuple containing the preprocessed input sequence (X) and output sequence (y).
    """
    # Select data for Jun, Jul, Aug, Sept
    sequence = sequence.sel(time=slice("1998-01-01","2016-12-31"))
    sequence = sequence.where(sequence.time.dt.month.isin([6,7,8,9]), drop=True)
    max = sequence.max()
    min = sequence.min()
    
    # Max-min normalization
    sequence = (sequence-min)/(max - min)
    sequence = np.array(sequence)
    
    # Exponential Space Transform
    valid_pts = np.where(sequence==sequence)
    sequence[valid_pts] = np.exp(sequence[valid_pts])
    sequence[valid_pts] = np.power(sequence[valid_pts],7)
    invalid_pts = np.where(sequence != sequence)
    sequence[invalid_pts] = 0
    
    # Prepare input and output sequences
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix + lead_time > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix + lead_time]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


def create_input(channels, n_steps):
    """
    Function to create the input for the model.
    
    Parameters:
        channels (list): List of channels to be used in the input.
        n_steps (int): Number of time steps in the input sequence.
        
    Returns:
        numpy.ndarray: The input for the model.
    """
    stack = []
    for i in range(len(channels)):
        channel_data = xr.open_dataarray(f"/home/cccr/roxy/matin/MTech_project/data/{channels[i]}")
        input, _ = preprocess_data(channel_data, n_steps=n_steps, channels=channels[i])
        input = np.expand_dims(input, axis=2)
        print(f"Adding channel {i} with shape: {input.shape}")
        stack.append(input)
    out = np.dstack(stack)
    return out


channels = ["FilteredrfBOB_13lag.nc","FilteredSSTBOB_13.nc"]
steps = 7

# Create input for the model
input = create_input(channels, steps)

# Preprocess target data
channel_data = xr.open_dataarray(add_data + "FilteredrfBOB_13lag.nc")
_, target = preprocess_data(channel_data, n_steps=steps, channels=channels[-1])
target = np.expand_dims(target, axis=1)

# Roll axes for input and target
x = np.rollaxis(input, 4, 2)
x = np.rollaxis(x, 4, 2)

y = np.rollaxis(target, 3, 1)
y = np.rollaxis(y, 3, 1)

# Assert that input and target shapes are consistent
assert input.shape[0] == target.shape[0]
assert input.shape[-1] == target.shape[-1]
assert input.shape[-2] == target.shape[-2]


# Print shape of input and target
print("INPUT SHAPE --> ", input.shape)
print("TARGET SHAPE --> ", target.shape)

# Delete original input and target variables to free up memory
del input
del target

# Perform additional processing or modeling steps as needed
seq = tf.keras.Sequential()

# Add layers to the model
seq.add(ConvLSTM2D(filters=4, kernel_size=(3,3), padding='same', input_shape=(7,25,39,2), return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=8, kernel_size=(3,3), padding='same', return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=8, kernel_size=(3,3), padding='same', return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=16, kernel_size=(3,3), padding='same', return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=16, kernel_size=(3,3), padding='same', return_sequences=False, data_format='channels_last'))
seq.add(Conv2D(filters=15, kernel_size=(3,3), activation='relu', padding='same', data_format='channels_last'))
seq.add(Conv2D(filters=1, kernel_size=(3,3), activation='relu', padding='same', data_format='channels_last'))
# Create the query, value, and key inputs
query_input = Input(shape=(25,39,2), name='query_input')
value_input = Input(shape=(25,39,2), name='value_input')
key_input = Input(shape=(25,39,2), name='key_input')

# Pass the inputs to the Attention layer
attention_output = Attention(name='attention')([query_input, value_input, key_input])
Adam = tf.keras.optimizers.Adam(learning_rate=10**-4)
seq.compile(loss='mean_squared_error', optimizer=Adam, metrics=['mae'])

print(seq.summary())

# Define the number of training epochs
n_epochs = 200
logdir = save_path
# Define the TensorBoard callback for monitoring training progress
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=10, batch_size=10, write_graph=True)

# Define the ReduceLROnPlateau callback for reducing the learning rate when the model plateaus
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=20, min_lr=10**-20)

# Define the ModelCheckpoint callback for saving the best model based on validation mean absolute error
checkpoint_callback_mae = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_mae',
                                                             mode='min', save_best_only=True)

# Define the ModelCheckpoint callback for saving the best model based on validation loss
checkpoint_callback_loss = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss',
                                                              mode='min', save_best_only=True)

# Define the EarlyStopping callback for stopping training when the model stops improving
early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)

# Define the TerminateOnNaN callback for terminating the training process if there is NaN value in any of the output
terminate_callback = tf.keras.callbacks.TerminateOnNaN()

# Train the model using the defined callbacks
history = seq.fit(x, y, epochs=n_epochs, validation_split=0.1,
                  batch_size=50, callbacks=[early_stop_callback,
                  checkpoint_callback_mae, checkpoint_callback_loss,
                  tb_callback, lr_callback, terminate_callback])
#Save the trained model
seq.save(model_path)

#Save the training history

np.save(f'{save_path}ConvLstm_sst13rf_rf_1bob.npy', history.history)               
              

Adding channel 0 with shape: (2310, 7, 1, 25, 39)
Adding channel 1 with shape: (2310, 7, 1, 25, 39)
INPUT SHAPE -->  (2310, 7, 2, 25, 39)
TARGET SHAPE -->  (2310, 1, 25, 39)


2023-03-14 16:32:43.761138: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 7, 25, 39, 4)      880       
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 7, 25, 39, 8)      3488      
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 7, 25, 39, 8)      4640      
                                                                 
 conv_lstm2d_3 (ConvLSTM2D)  (None, 7, 25, 39, 16)     13888     
                                                                 
 conv_lstm2d_4 (ConvLSTM2D)  (None, 25, 39, 16)        18496     
                                                                 
 conv2d (Conv2D)             (None, 25, 39, 15)        2175      
                                                                 
 conv2d_1 (Conv2D)           (None, 25, 39, 1)         1

Epoch 44/200
42/42 [==============================] - 249s 6s/step - loss: 1039.0618 - mae: 20.9487 - val_loss: 925.3461 - val_mae: 19.1659 - lr: 1.0000e-04
Epoch 45/200
42/42 [==============================] - 259s 6s/step - loss: 1038.5737 - mae: 20.8772 - val_loss: 925.0869 - val_mae: 19.1531 - lr: 1.0000e-04
Epoch 46/200
42/42 [==============================] - 257s 6s/step - loss: 1038.3241 - mae: 20.8963 - val_loss: 925.1987 - val_mae: 19.2196 - lr: 1.0000e-04
Epoch 47/200
42/42 [==============================] - 243s 6s/step - loss: 1038.3600 - mae: 21.0069 - val_loss: 924.7499 - val_mae: 19.1706 - lr: 1.0000e-04
Epoch 48/200
42/42 [==============================] - 260s 6s/step - loss: 1038.1094 - mae: 20.8671 - val_loss: 924.2600 - val_mae: 19.0878 - lr: 1.0000e-04
Epoch 49/200
42/42 [==============================] - 250s 6s/step - loss: 1037.8590 - mae: 20.9485 - val_loss: 924.3668 - val_mae: 19.1779 - lr: 1.0000e-04
Epoch 50/200
42/42 [==============================] - 260s

42/42 [==============================] - 253s 6s/step - loss: 1028.0309 - mae: 20.7368 - val_loss: 914.3483 - val_mae: 18.9222 - lr: 1.0000e-04
Epoch 97/200
42/42 [==============================] - 251s 6s/step - loss: 1027.7628 - mae: 20.7757 - val_loss: 914.3943 - val_mae: 18.9853 - lr: 1.0000e-04
Epoch 98/200
42/42 [==============================] - 250s 6s/step - loss: 1027.6295 - mae: 20.8006 - val_loss: 914.0654 - val_mae: 18.9574 - lr: 1.0000e-04
Epoch 99/200
42/42 [==============================] - 264s 6s/step - loss: 1027.2653 - mae: 20.7407 - val_loss: 914.0493 - val_mae: 18.9971 - lr: 1.0000e-04
Epoch 100/200
42/42 [==============================] - 261s 6s/step - loss: 1027.1191 - mae: 20.7326 - val_loss: 913.7040 - val_mae: 18.9535 - lr: 1.0000e-04
Epoch 101/200
42/42 [==============================] - 265s 6s/step - loss: 1027.0133 - mae: 20.7644 - val_loss: 913.5229 - val_mae: 18.9542 - lr: 1.0000e-04
Epoch 102/200
42/42 [==============================] - 256s 6s/step -

Epoch 148/200
42/42 [==============================] - 244s 6s/step - loss: 588.8544 - mae: 12.9000 - val_loss: 486.1426 - val_mae: 11.0074 - lr: 1.0000e-04
Epoch 149/200
42/42 [==============================] - 261s 6s/step - loss: 511.6427 - mae: 11.7918 - val_loss: 411.6338 - val_mae: 9.6531 - lr: 1.0000e-04
Epoch 150/200
42/42 [==============================] - 257s 6s/step - loss: 439.6091 - mae: 10.5529 - val_loss: 363.8122 - val_mae: 8.6517 - lr: 1.0000e-04
Epoch 151/200
42/42 [==============================] - 262s 6s/step - loss: 396.7207 - mae: 9.7788 - val_loss: 329.9468 - val_mae: 8.1928 - lr: 1.0000e-04
Epoch 152/200
42/42 [==============================] - 261s 6s/step - loss: 367.3115 - mae: 9.3416 - val_loss: 308.5773 - val_mae: 7.7276 - lr: 1.0000e-04
Epoch 153/200
42/42 [==============================] - 262s 6s/step - loss: 344.1711 - mae: 9.0014 - val_loss: 285.7401 - val_mae: 7.5523 - lr: 1.0000e-04
Epoch 154/200
42/42 [==============================] - 257s 6s/ste

In [26]:
def preprocess_data(sequence, n_steps, channels):
    """
    Function to preprocess the data and prepare it for model training.
    
    Parameters:
        sequence (xarray.Dataset): Data sequence to be preprocessed.
        n_steps (int): Number of time steps in the input sequence.
        channels (int): Number of channels in the input sequence.
    
    Returns:
        tuple: A tuple containing the preprocessed input sequence (X) and output sequence (y).
    """
    # Select data for Jun, Jul, Aug, Sept
    sequence = sequence.sel(time=slice("2017-08-01","2017-08-9"))
    sequence = sequence.where(sequence.time.dt.month.isin([6,7,8,9]), drop=True)
    max = sequence.max()
    min = sequence.min()
    
    # Max-min normalization
    sequence = (sequence-min)/(max - min)
    sequence = np.array(sequence)
    
    # Exponential Space Transform
    valid_pts = np.where(sequence==sequence)
    sequence[valid_pts] = np.exp(sequence[valid_pts])
    sequence[valid_pts] = np.power(sequence[valid_pts],7)
    invalid_pts = np.where(sequence != sequence)
    sequence[invalid_pts] = 0
    
    # Prepare input and output sequences
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix + lead_time > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix + lead_time]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


def create_input(channels, n_steps):
    """
    Function to create the input for the model.
    
    Parameters:
        channels (list): List of channels to be used in the input.
        n_steps (int): Number of time steps in the input sequence.
        
    Returns:
        numpy.ndarray: The input for the model.
    """
    stack = []
    for i in range(len(channels)):
        channel_data = xr.open_dataarray(f"/home/cccr/roxy/matin/MTech_project/data/{channels[i]}")
        input, _ = preprocess_data(channel_data, n_steps=n_steps, channels=channels[i])
        input = np.expand_dims(input, axis=2)
        print(f"Adding channel {i} with shape: {input.shape}")
        stack.append(input)
    out = np.dstack(stack)
    return out


channels = ["FilteredrfBOB_13lag.nc","FilteredSSTBOB_13.nc"]
steps = 7

# Create input for the model
input = create_input(channels, steps)

# Preprocess target data
channel_data = xr.open_dataarray(add_data + "FilteredrfBOB_13lag.nc")
_, target = preprocess_data(channel_data, n_steps=steps, channels=channels[-1])
target = np.expand_dims(target, axis=1)

# Roll axes for input and target
x = np.rollaxis(input, 4, 2)
x = np.rollaxis(x, 4, 2)

y = np.rollaxis(target, 3, 1)
y = np.rollaxis(y, 3, 1)

# Assert that input and target shapes are consistent
assert input.shape[0] == target.shape[0]
assert input.shape[-1] == target.shape[-1]
assert input.shape[-2] == target.shape[-2]


# Print shape of input and target
print("INPUT SHAPE --> ", input.shape)
print("TARGET SHAPE --> ", target.shape)

# Delete original input and target variables to free up memory
del input
del target


Adding channel 0 with shape: (1, 7, 1, 25, 39)
Adding channel 1 with shape: (1, 7, 1, 25, 39)
INPUT SHAPE -->  (1, 7, 2, 25, 39)
TARGET SHAPE -->  (1, 1, 25, 39)


In [35]:
x[0,:,:,:,0].shape

(7, 25, 39)

In [38]:
y[:,:,:,0].shape

(1, 25, 39)

In [45]:
xr.DataArray(y[:,:,:,0], dim = [], coords = {'time':})

<xarray.DataArray (dim_0: 1, dim_1: 25, dim_2: 39)>
array([[[1.35847545e+02, 1.57354806e+02, 1.66031650e+02, 1.79101410e+02,
         2.28960544e+02, 2.13255130e+02, 2.04676394e+02, 1.53013385e+02,
         9.51180030e+01, 4.09213778e+01, 3.17883954e+01, 4.53971471e+01,
         4.62357442e+01, 4.15999846e+01, 4.68992632e+01, 5.43378952e+01,
         2.81801202e+01, 2.18429448e+01, 1.53738122e+01, 1.75753138e+01,
         1.61623108e+01, 2.18808322e+01, 3.15545633e+01, 5.93994192e+01,
         4.96874100e+01, 6.36613391e+01, 6.59871291e+01, 7.58355698e+01,
         8.07320088e+01, 8.17825639e+01, 6.01025553e+01, 4.64610767e+01,
         6.19311687e+01, 6.93843873e+01, 4.05975803e+01, 3.77170480e+01,
         2.24613014e+01, 1.82863371e+01, 1.17856209e+01],
        [1.38926352e+02, 1.46723888e+02, 1.12101164e+02, 1.31353428e+02,
         1.40760804e+02, 1.30545132e+02, 1.04389473e+02, 9.33810693e+01,
         1.43637526e+02, 9.02550406e+01, 7.38155802e+01, 8.27772829e+01,
         1.22136907e+02, 1.06603881e+02, 7.16537442e+01, 5.83160355e+01,
         3.65759624e+01, 2.11275438e+01, 2.42621392e+01, 1.61517898e+01,
         1.24375883e+01, 2.22808236e+01, 2.59286467e+01, 2.54080374e+01,
         5.03986104e+01, 5.33554625e+01, 6.38909681e+01, 4.91398794e+01,
         5.10526510e+01, 6.55493200e+01, 6.96699121e+01, 5.10167217e+01,
         5.89864848e+01, 4.33060287e+01, 3.53197947e+01, 4.42765058e+01,
         5.29659079e+01, 4.18582515e+01, 3.85531690e+01],
...
        [7.41742008e+01, 3.31903555e+01, 3.31972118e+01, 2.04138940e+01,
         3.25234484e+01, 1.77109757e+01, 1.71300712e+01, 1.10235240e+01,
         3.10613840e+01, 5.61929669e+01, 1.76055098e+01, 8.09568524e+00,
         2.14944419e+01, 2.34552703e+01, 4.31334131e+01, 2.22285060e+01,
         1.27028814e+01, 4.96332920e+00, 3.70684008e+00, 4.83073693e+00,
         3.32793544e+00, 3.05130061e+00, 5.86587364e+00, 1.20827968e+01,
         1.09569482e+01, 6.12699287e+00, 1.15866971e+01, 3.27708981e+01,
         2.82883300e+01, 5.35825047e+01, 3.21068602e+01, 3.76652909e+01,
         7.84126313e+01, 1.22723053e+02, 8.87864643e+01, 5.81802785e+01,
         1.41075094e+01, 3.28882949e+00, 2.65129036e+00],
        [2.71828566e+01, 3.20274814e+01, 4.25441559e+01, 2.68761284e+01,
         5.12302792e+01, 3.22575332e+01, 2.97150616e+01, 1.47433606e+01,
         3.77001564e+01, 6.50146153e+01, 2.50374677e+01, 1.20286386e+01,
         5.77599530e+00, 1.39124663e+01, 1.25741930e+01, 7.32401661e+00,
         2.05140636e+01, 1.01289997e+01, 6.25532093e+00, 5.07399910e+00,
         5.22110920e+00, 4.66501150e+00, 3.40273060e+00, 3.31397829e+00,
         4.46466152e+00, 3.71179454e+00, 6.60152666e+00, 1.40469031e+01,
         3.35349893e+01, 5.14641598e+01, 4.24848044e+01, 3.74958918e+01,
         4.93223915e+01, 5.03291973e+01, 2.84086010e+01, 1.35212662e+01,
         4.83961984e+00, 3.54219286e+00, 3.76498103e+00]]])
Dimensions without coordinates: dim_0, dim_1, dim_2

In [40]:
pred[:,:,:,0].shape

(1, 25, 39)

In [42]:
xr.open_dataarray(add_data + "FilteredrfBOB_13lag.nc").sel(time=slice("2017-08-01","2017-08-9"))

<xarray.DataArray (time: 9, lat: 25, lon: 39)>
array([[[-0.374979, -0.412089, ..., -0.068898, -0.146327],
        [-0.249327, -0.307384, ..., -0.195054, -0.184903],
        ...,
        [ 0.031834,  0.057187, ..., -0.213189, -0.289413],
        [-0.203384, -0.004878, ..., -0.182432, -0.255864]],

       [[-0.33074 , -0.359739, ..., -0.112893, -0.198165],
        [-0.215506, -0.267521, ..., -0.204382, -0.196652],
        ...,
        [ 0.020136,  0.017922, ..., -0.310456, -0.387939],
        [-0.231829, -0.039427, ..., -0.273823, -0.338335]],

       ...,

       [[ 0.05215 ,  0.073052, ..., -0.24817 , -0.334011],
        [ 0.07236 ,  0.072909, ..., -0.127061, -0.138216],
        ...,
        [-0.001454, -0.134571, ..., -0.568905, -0.616934],
        [-0.207417, -0.146829, ..., -0.546683, -0.545159]],

       [[ 0.11444 ,  0.140714, ..., -0.244056, -0.322584],
        [ 0.118447,  0.128209, ..., -0.096013, -0.110716],
        ...,
        [ 0.006265, -0.137492, ..., -0.550754, -0.589276],
        [-0.173187, -0.143868, ..., -0.537487, -0.526582]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2017-08-01 2017-08-02 ... 2017-08-09
  * lon      (lon) float32 84.12 84.38 84.62 84.88 ... 92.88 93.12 93.38 93.62
  * lat      (lat) float32 15.12 15.38 15.62 15.88 ... 20.38 20.62 20.88 21.12

In [25]:
model = tf.keras.models.load_model(model_path)
pred = model.predict(x)

actual = y.flatten()
prediction = pred.flatten()
corr = np.corrcoef(actual,prediction)
corr

1/1 [==============================] - 1s 858ms/step


array([[1.       , 0.8111932],
       [0.8111932, 1.       ]])

In [5]:
pred.shape

(236, 25, 39, 1)

In [7]:
x.shape

(236, 7, 25, 39, 2)

In [8]:
x[:,:,:,:,0]

array([[[[ 31.79536234,  38.81434563,  41.29574909, ...,  40.69823528,
           47.04685876,  49.97203383],
         [ 24.50496202,  30.92650356,  32.93196006, ...,  44.16181174,
           45.5945786 ,  48.76934417],
         [ 24.20133923,  23.39000929,  26.11703694, ...,  34.33401396,
           39.51700828,  42.8174556 ],
         ...,
         [ 30.92899035,  34.25304279,  33.01897753, ...,  21.67897161,
           23.99302841,  20.11429632],
         [ 28.26028908,  30.55385174,  29.4099596 , ...,  35.30787191,
           34.44097029,  26.28781937],
         [ 40.05643783,  35.41967401,  32.45584662, ...,  39.21768356,
           29.50326353,  32.01185571]],

        [[ 34.64485838,  43.43785675,  46.55544729, ...,  39.81628874,
           47.99385051,  51.5997279 ],
         [ 26.39828926,  33.88586666,  35.48759136, ...,  43.96718453,
           45.34398192,  49.10362902],
         [ 25.21826787,  23.87541935,  27.33029204, ...,  34.34355338,
           38.64061213,  41.49723

In [11]:
x[:,:,:,:,1]

array([[[[ 42.60941317,  45.06608116,  53.30995288, ...,  36.02027705,
           47.65624455,   0.        ],
         [ 50.40389024,  53.30995288,  66.68633104, ...,  38.09704495,
           47.65624455,   0.        ],
         [ 56.37393103,  66.68633104,  78.89864567, ...,  38.09704495,
           45.06608116,  53.30995288],
         ...,
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ],
         [  0.        ,   0.        ,   0.        , ...,   0.        ,
            0.        ,   0.        ]],

        [[ 38.09704495,  42.60941317,  50.40389024, ...,  34.05671911,
           42.60941317,   0.        ],
         [ 42.60941317,  47.65624455,  59.62419949, ...,  34.05671911,
           42.60941317,   0.        ],
         [ 50.40389024,  59.62419949,  70.53116631, ...,  34.05671911,
           40.28666447,  47.65624